In [2]:
import numpy as np
import time
import re
import collections
import datetime
import math
from scipy.stats import binom

In [3]:
class Pileup_line:
    def __init__(self, line): 
        split_line = line.split("\t")
        self.sequence = split_line[0]
        self.position = split_line[1]
        self.ref_base = split_line[2]
        self.read_count = split_line[3]
        self.read_results = split_line[4]
        self.base_quality = split_line[5]
        self.mapping_quality = split_line[6]
    
    # Get sorted list of most common characters from string read_result
    def ideal_rr_count(self, read_result):
        # Convert ',' to '.' and lower to upper case
        read_result = read_result.replace(',','.').upper()
        char_counter = collections.Counter(read_result).most_common() 
        var_list = char_counter 
        return var_list

    # Function for segregating indels from string read_result.
    # Output of the function are two lists:
    # 1) list of '.', ',', and single chars
    # 2) list of insertions or deletions or both
    def segregate_indels(self, read_result):
        # Find numbers, because both insertion and deletion sequences have them
        numbers = np.array([int(m) for m in re.findall('[\d]+', read_result)])
        # Indices of numbers
        indices = np.array([m.span() for m in re.finditer('[\d]+', read_result)])
        # String -> np.array
        read_result = np.array([char for char in read_result])
        # ranges is a list of tuples. Number of tuples is the same as number of INDELs in read_result.
        # First number in each tuple is the index of the beginning of the INDEL, which is alway either '.' or ','. Second number is the index of the last element of an INDEL.
        ranges = [(index[0]-2,index[1]+numbers[i]) for i,index in enumerate(indices)]
        # indels_list is list of indels on which can be applied collection.counter
        indels_list = []
        for item in ranges:
            indels_list.append(''.join(read_result[item[0]:item[1]]))

        # Deleting INDELs from read_result
        indices_for_removing = []
        for item in ranges:
            indices_for_removing.extend([*range(item[0],item[1])])
        read_result = np.delete(read_result,indices_for_removing)
        # read_result to string
        read_result = ''.join(read_result[:])
        
        return read_result, indels_list

    # Delete '^' and the following character
    def delete_carets(self, read_result):
        read_result = re.sub('\^.','',read_result)
        return read_result

    # Output of the function are two most common variants in self.read_results.
    # Variants are represented with the Variant class.
    def determine_variant(self):
        read_result = self.read_results
        read_count = self.read_count
        
        ins = read_result.find("+") # insertion flag
        dele = read_result.find("-") # deletion flag
        car = read_result.find("^") # ^ flag
  
        # No INDELs
        if ins == -1 and dele == -1:
            if car != -1:
                read_result = self.delete_carets(read_result)
            read_result = re.sub('\$|\*','',read_result)
            list_of_vars = self.ideal_rr_count(read_result)
            if len(list_of_vars) == 0:
                Variant1 = None
                Variant2 = None
            elif len(list_of_vars) == 1:
                Variant1 = Variant(list_of_vars[0][0],list_of_vars[0][1], self.read_count)
                Variant2 = None
            else:
                Variant1 = Variant(list_of_vars[0][0],list_of_vars[0][1], self.read_count)
                Variant2 = Variant(list_of_vars[1][0],list_of_vars[1][1], self.read_count)

        # INDELs are present
        elif ins != -1 or dele != -1:
            if car != -1:
                read_result = self.delete_carets(read_result)
            read_result = re.sub('\$|\*','',read_result)
            read_result, indels_list = self.segregate_indels(read_result)

            # INDELS_LIST
            # ',' -> '.', lower case -> UPPER CASE
            indels_list = [s.replace(',','.') for s in indels_list]
            number_of_insertions = collections.Counter(map(str.upper, indels_list)).most_common()
            # READ_RESULT
            list_of_vars = self.ideal_rr_count(read_result)
            list_of_vars.extend(number_of_insertions)
            list_of_vars = sorted(list_of_vars, key=lambda tup: tup[1], reverse = True)
      
            if len(list_of_vars) == 0:
                Variant1 = None
                Variant2 = None
            elif len(list_of_vars) == 1:
                Variant1 = Variant(list_of_vars[0][0],list_of_vars[0][1], self.read_count)
                Variant2 = None
            else:
                Variant1 = Variant(list_of_vars[0][0],list_of_vars[0][1], self.read_count)
                Variant2 = Variant(list_of_vars[1][0],list_of_vars[1][1], self.read_count)
            
        return Variant1, Variant2
            
    # Input of determine_genotype are variants outputed from determine_variant().
    # Outputs are gonotype and genotype_list.
    # Genotype = {'0/0', '0/1', '1/1', '1/2'}
    # Genotype_list depends on the genotype; if the genotype is '0/0', genotype_list is empty;
    # if the genotype is '0/1' or '1/1' genotype_list consists of a single element, which is
    # the corresponding Variant; and if the genotype is '1/2' the lists consists of both variants.
    def determine_genotype(self,variant1, variant2):
        genotype_list = []
        if variant1 is None:
            return '0/0', []
        if variant2 is None:
            if variant1.variant_type == 'REF':
                genotype = '0/0'
            else:
                genotype = '0/1'
                genotype_list.append(variant1)
            return genotype, genotype_list
            
        k1 = variant1.number_of_supporting_reads
        k2 = variant2.number_of_supporting_reads
        n = k1+k2
        p = 0.95 # probability of success
        P_k1k1 = binom.pmf(k1,n,p)
        P_k1k2 = binom.pmf(n,n,p)
        P_k2k2 = binom.pmf(k2,n,p)
        # Depending on the genotype, the ALT field is later (in the main function) formed:
        # 1) 0/0 --> there is no ALT
        # 2) 1/1 or 0/1 --> one ALT
        # 3) 1/2 --> two ALTs
        # The assumption is that there is not going to be more than 2 ALTs
        if P_k1k1 >= P_k1k2 and P_k1k1 >= P_k2k2: # genotype = 'k1/k1'
            if variant1.variant_type == 'REF':
                genotype = '0/0'
            else:
                genotype = '1/1'
                genotype_list.append(variant1)
        elif P_k1k2 >= P_k1k1 and P_k1k2 >= P_k2k2: # genotype = 'k1/k2'
            if variant1.variant_type == 'REF' or variant2.variant_type == 'REF':
                genotype = '0/1'
                if variant1.variant_type == 'REF':
                    genotype_list.append(variant2)
                else:
                    genotype_list.append(variant1)
            else:
                genotype = '1/2'
                genotype_list.append(variant1)
                genotype_list.append(variant2)
        #elif P_k1k1 == P_k1k2 or P_k1k1 == P_k2k2 or P_k1k2 == P_k2k2:   
        else: # genotype = 'k2/k2'
            if variant2.variant_type == 'REF':
                genotype = '0/0'
            else:
                genotype = '1/1'
                genotype_list.append(variant2)

        return genotype, genotype_list
        

In [4]:
class Variant:

    def __init__(self, sequence, number_of_supporting_reads, total_num_of_reads): 
        self.variant_type, self.variant_sequence, self.variant_len = self.convert_pileup_string(sequence)
        self.number_of_supporting_reads = number_of_supporting_reads
        self.VAF = float(number_of_supporting_reads)/float(total_num_of_reads)

    # Input of the function is string for this variant from pileup.
    # Outputs of the function are variant_type, variant_sequence and variant_len
    # variant_type = {'SNV', 'insertion', 'deletion', 'REF'}
    # variant_sequence = '.' for REFs and upper case string for SNVs and INDELS, i.e. 'A' for SNV, 'AG' for INDELs
    def convert_pileup_string(self, sequence):
        if len(sequence) == 1:
            variant_sequence = sequence
            variant_len = 1
            if sequence == '.': 
                variant_type = 'REF'
            else:
                variant_type = 'SNV'
        else:
            if sequence[1] == '+':
                variant_type = 'insertion'
            else:
                variant_type = 'deletion'
            # convert string that looks like this: ,+3AGC  ...to this: AGC
            sequence = sequence[2:]
            variant_sequence = ''.join(filter(str.isalpha, sequence))
            variant_len = int(''.join(filter(str.isdigit, sequence)))
        return variant_type, variant_sequence, variant_len
        

In [14]:
with open('/sbgenomics/project-files/merged-normal_pileup.pileup', 'r') as pileup_file, open('vcf4-2garbage.txt', 'w') as vcf:
    vcf = create_vcf_header(vcf)
    start = time.time()
    for line in iter(pileup_file.readline, ''):
        position = pileup_file.tell() # take the current cursor position
        pileup = Pileup_line(line)
        # If there are no reads at this position, we assume that there are no variants
        if pileup.read_count == '0':
            continue
        variant1, variant2 = pileup.determine_variant()
        genotype, genotype_list = pileup.determine_genotype(variant1, variant2)
        # variants are INDEL --> indel_string = 'INDEL'
        # variants are not INDEL --> indel_string = '.'
        indel_string = '.'
            
        # If the genotype is '0/0', there is no variant
        if genotype == '0/0':
            continue
        elif genotype == '1/1' or genotype == '0/1':
            variant = genotype_list[0]
            VAF = variant.VAF
            if variant.variant_type == 'insertion':
                REF = pileup.ref_base
                ALT = pileup.ref_base + variant.variant_sequence
            elif variant.variant_type == 'deletion':
                REF = pileup.ref_base
                ALT = pileup.ref_base
            else: # SNV
                REF = pileup.ref_base
                ALT = variant.variant_sequence
        
            if variant.variant_type == 'insertion' or variant.variant_type == 'deletion':
                indel_string = 'INDEL'
                condition = True
                del_temp = False # del_temp variable is needed only in case of deletion
                if variant.variant_type == 'deletion':
                    del_temp = True
                while(condition):
                    for i in range(variant.variant_len):
                        line = pileup_file.readline()
                        pileup_temp = Pileup_line(line)
                        if variant.variant_sequence[i] == pileup_temp.ref_base:
                            REF += pileup_temp.ref_base
                            if not del_temp:
                                ALT += pileup_temp.ref_base
                        else:
                            condition = False
                            # The cursor does not go back to variable position in the case of deletion
                            # because we want to avoid unnecessary '*'
                            if variant.variant_type == 'insertion': 
                                pileup_file.seek(position)
                            break
                    del_temp = False
            vcf.write(pileup.sequence + '\t' + pileup.position + '\t' + '.' + '\t' + REF + '\t' + ALT + '\t' + '.' + '\t' + '.' + '\t' + indel_string + '\t' + 'GT:VAF' + '\t' + genotype + ':' + str(VAF) + '\n')
        else: # genotype == '1/2'
            refs = ['','']
            alts = ['','']
            vafs = ['','']
            indel_string = ['.','.']
            for j in range(2):
                variant = genotype_list[j]
                vafs[j] += str(variant.VAF)
                if variant.variant_type == 'insertion':
                    refs[j] += pileup.ref_base
                    alts[j] += pileup.ref_base + variant.variant_sequence
                elif variant.variant_type == 'deletion':
                    refs[j] += pileup.ref_base
                    alts[j] += pileup.ref_base
                else: # SNV
                    refs[j] += pileup.ref_base
                    alts[j] += variant.variant_sequence
        
                if variant.variant_type == 'insertion' or variant.variant_type == 'deletion':
                    condition = True
                    indel_string[j] = 'INDEL'
                    del_temp = False # del_temp variable is needed only in case of deletion
                    if variant.variant_type == 'deletion':
                        del_temp = True
                    while(condition):
                        for i in range(variant.variant_len):
                            line = pileup_file.readline()
                            pileup_temp = Pileup_line(line)
                            if variant.variant_sequence[i] == pileup_temp.ref_base:
                                refs[j] += pileup_temp.ref_base
                                if not del_temp:
                                    alts[j] += pileup_temp.ref_base
                            else:
                                condition = False
                                # The cursor does not go back to variable position in the case of
                                # deletion because we want to avoid unnecessary '*'
                                if variant.variant_type == 'insertion' or j == 0: 
                                    pileup_file.seek(position)
                                break
                        del_temp = False # flag for first pass of while loop
            
            if refs[0] == refs[1]:
                REF = refs[0]
                ALT = alts[0]+','+alts[1]
                vcf.write(pileup.sequence + '\t' + pileup.position + '\t' + '.' + '\t' + REF + '\t' + ALT + '\t' + '.' + '\t' + '.' + '\t' + indel_string[0] + ',' + indel_string[1] + '\t' + 'GT:VAF' + '\t' + genotype + ':' + vafs[0]+','+vafs[1] + '\n')
            else:
                vcf.write(pileup.sequence + '\t' + pileup.position + '\t' + '.' + '\t' + refs[0] + '\t' + alts[0] + '\t' + '.' + '\t' + '.' + '\t' + indel_string[0] + '\t' + 'GT:VAF' + '\t' + genotype + ':' + vafs[0] + '\n')
                vcf.write(pileup.sequence + '\t' + pileup.position + '\t' + '.' + '\t' + refs[1] + '\t' + alts[1] + '\t' + '.' + '\t' + '.' + '\t' + indel_string[1] + '\t' + 'GT:VAF' + '\t' + genotype + ':' + vafs[1] + '\n')
                
    end = time.time()
    print(end-start)

191.09894514083862


In [4]:
def create_vcf_header(vcf):
    x = datetime.datetime.now()
    fileformat = '##fileformat=VCFv4.2'
    date = x.strftime('%Y%m%d')
    source = '##source=Jadranka&Suki'
    filedate = '##fileDate='+date
    reference = '##reference=file:///sbgenomics/Projects/4dc3fffd-51c3-42f3-885b-8b18384be984/human_g1k_v37_decoy.fasta'
    vcf.write(fileformat+'\n' + filedate + '\n' + source + '\n' + reference + '\n')
    with open('/sbgenomics/project-files/human_g1k_v37_decoy.fasta.fai','r') as fai:
        for line in fai:
            split_line = line.split("\t")
            ID = split_line[0]
            length = split_line[1]
            contig = '##contig=<ID=' + str(ID) + ', length=' + str(length) + '>'
            vcf.write(contig + '\n')
    fai.close()
    filter = '##FILTER=<ID=PASS,Description="All filters passed">'
    vcf.write(filter + '\n')
    vcf.write('#CHROM' + '\t' + 'POS' + '\t' + 'ID' + '\t' + 'REF' + '\t' + 'ALT' + '\t' + 'QUAL' + '\t' + 'FILTER' + '\t' + 'INFO' + '\t' + 'FORMAT' + '\t' + 'HCC1143BL' + '\n')
    return vcf

In [12]:
assert pileup.segregate_indels('...+1Aa,,,,-2gc....') == ('..a,,,....', ['.+1A', ',-2gc'])
assert pileup.delete_carets('...^|,,^]$..^.') == '...,,$..'
assert pileup.ideal_rr_count('...,,,AAGGgg..Tccccc,,,$$**..**atttttttC,,') == [('.', 15), ('T', 8), ('C', 6), ('G', 4), ('*', 4), ('A', 3), ('$', 2)]


variant_1 = Variant('A', 2, 10)
#variant_2 = Variant(sequence, number_of_supporting_reads, total_num_of_reads)
assert variant_1.convert_pileup_string(',+4GCCT') == ('insertion', 'GCCT', 4)
assert variant_1.convert_pileup_string(',-1A') == ('deletion','A',1)
assert variant_1.convert_pileup_string('.') ==('REF','.',1)
assert variant_1.convert_pileup_string('A') == ('SNV','A',1)